In [5]:
#3.1 텐서와 그래프 실행
import tensorflow as tf
import numpy as np

In [6]:
hello = tf.constant('Hello, TensorFlow!')
print(hello)

Tensor("Const:0", shape=(), dtype=string)


In [7]:
a = tf.constant(10)
b = tf.constant(32)
c = tf.add(a,b)
print(c)

Tensor("Add:0", shape=(), dtype=int32)


In [8]:
sess = tf.Session()

print(sess.run(hello))
print(sess.run([a, b, c]))

sess.close()

b'Hello, TensorFlow!'
[10, 32, 42]


In [2]:
import tensorflow as tf

In [3]:
import numpy as np

In [10]:
3 # 랭크가 0인 텐서; 셰이프는 []
[1., 2., 3.] # 랭크가 1인 텐서; 셰이프는 [3]
[[1., 2., 3], [4., 5., 6.]] # 랭크가 2인 텐서; 셰이프는 [2, 3]
[[[1., 2., 3.]], [[7., 8., 9.]]] # 랭크가 3인 텐서; 셰이프는 [2, 1, 3]

#랭크는 차원의 수(0-스칼라, 1-벡터, 2-행렬, 3이상-n차원 텐서), 셰이프는 각 차원의 요소 

[[[1.0, 2.0, 3.0]], [[7.0, 8.0, 9.0]]]

In [11]:
# 3.2 플레이스홀더와 변수
# None : 크기가 정해지지 않았다는 것을 의미함.
# placeholder? 그래프에 사용할 입력값을 나중에 받기 위해 사용하는 매개변수(parameter)
X = tf.placeholder(tf.float32, [None, 3])
print(X)

Tensor("Placeholder_1:0", shape=(?, 3), dtype=float32)


In [12]:
# 나중에 플레이스홀더 X에 넣을 자료 정의. 텐서 모양이 (?, 3)이므로 두 번째 차원은 3개의 요소를 가짐.
x_data = [[1, 2, 3], [4, 5, 6]]

In [14]:
# Variable(변수) : 그래프를 최적화하는 용도. 
# tf.random_normal : normal distribution(정규분포)의 무작위 값으로 초기화.
W = tf.Variable(tf.random_normal([3, 2]))
b = tf.Variable(tf.random_normal([2, 1]))

# tf.matmul : X와 W가 행렬이므로. 아닐 경우 곱셈 연사자(*)나 tf.mul 함수 사용.
# 입력값과 변수들을 계산할 수식 
expr = tf.matmul(X, W) + b

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer()) # tf.global_Variables_initializer : 앞에서 정의한 변수 초기화하는 함수.

print("=== x_data ===")
print(x_data)
print("=== W ===")
print(sess.run(W))
print("=== b ===")
print(sess.run(b))
print("=== expr ===")
print(sess.run(expr, feed_dict={X: x_data})) #feed_dict 매개변수 : 그래프를 실행할 때 사용할 입력값 지정.

sess.close()
# X(x_data) = [2,3] W = [3,2] 결과값은 X의 행의 갯수와 W의 열의 갯수인 [2,2] 형태.

=== x_data ===
[[1, 2, 3], [4, 5, 6]]
=== W ===
[[-0.6866561   0.6524187 ]
 [-1.9785972  -2.0417154 ]
 [-0.52403843 -1.0505495 ]]
=== b ===
[[-0.8381433 ]
 [-0.23340197]]
=== expr ===
[[ -7.054109  -7.420804]
 [-16.017242 -14.135602]]


In [16]:
# 3.3 선형 회귀 모델 구현하기
# 선형회귀 : 주어진 x와 y 값으로 서로 간의 관계를 파악하는 것.

x_data = [1, 2, 3]
y_data = [1, 2, 3]

# 변수 W와 b를 각각 -1.0~1.0사이의 uniform distribution(균등분포)을 가진 무작위 값으로 초기화.
W = tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b = tf.Variable(tf.random_uniform([1], -1.0, 1.0))

#자료를 입력받을 플레이스홀더 설정. 각각의 이름을 지정해 놓음.
X = tf.placeholder(tf.float32, name="X")
Y = tf.placeholder(tf.float32, name="Y")

# X와 Y의 선형관계를 분석하기 위한 수식. 
# X가 주어졌을 때 Y를 만드는 W(Weight, 가중치)와 b(bias, 편향)를 찾는다는 뜻.
hypothesis = W * X + b

In [18]:
# loss function(손실 함수) : 손실값(실제값과 예측값의 차이)을 계산하는 함수.
# cost(비용) : 전체 데이터에 대해 구한 손실.(모든 데이터에 대한 손실값의 평균)
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# gradient descent(경사하강법) 최적화 함수를 이용해 손실값을 최소화하는 연산 그래프 생성.
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1) #learning rate(학습률) : 얼마나 '급하게' 할 것인가.
train_op = optimizer.minimize(cost)

In [43]:
#이제 그래프를 실행해서 학습을 시키고 결과를 보러 가볼까요?
# 파이썬의 with 기능을 이용, 세션 블록을 만들고 세션 종료를 자동처리.
with tf.Session() as sess: 
    sess.run(tf.global_variables_initializer()) # 변수 초기화.

In [44]:
 # train_op : 최적화 수행 그래프.
# 실행 시마다 변화하는 손실값을 출력.
#학습은 100번 수행, feed_dict 매개변수를 통해 상관관계를 알아내고자하는 데이터(X, Y) 입력.
for step in range(100):
    _, cost_val = sess.run([train_op, cost], feed_dict={X: x_data, Y: y_data})
    
    print(step, cost_val, sess.run(W), sess.run(b))

RuntimeError: Attempted to use a closed Session.

In [ ]:

print("X: 5, Y:", sess.run(hypothesis, feed_dict = {X: 5}))